Imports

In [18]:
import os 
import pandas as pd
import numpy as np 
from PIL import Image
import tensorflow as tf

Build our X and y with all the data we need 

In [2]:
directory = '../raw_data/faces/' 
folders = [x[0] for x in os.walk(directory)][1:]  #ignore the first folder which is the root dir

In [10]:
emotions = {'anger': 0, 
            'contempt': 1, 
            'disgust': 2,
            'fear': 3,
            'happiness': 4,
            'neutrality': 5,
            'sadness': 6, 
            'surprise': 7}

In [59]:
from tensorflow.keras.utils import to_categorical

ims = []
labels = []

for folder in folders: 
    for filename in os.listdir(folder): 
        emotion = os.path.basename(folder) #get folder name 
        f = os.path.join(folder, filename)
        # checking if it is a file
        if os.path.isfile(f):
            im = Image.open(f)
            im = np.expand_dims(np.array(im), axis=2)
            ims.append(im)
            emotion_enum = emotions[emotion]
            labels.append(emotion_enum)

In [73]:
num_classes = len(set(labels))
y = to_categorical(labels, num_classes)
X = np.array(ims)
p = np.random.permutation(len(X))
X, y = X[p], y[p]

first_split = int(len(X) /6.)
second_split = first_split + int(len(X) * 0.2)
X_test, X_val, X_train = X[:first_split], X[first_split:second_split], X[second_split:]
y_test, y_val, y_train = y[:first_split], y[first_split:second_split], y[second_split:]

In [69]:
y_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]], dtype=float32)

In [70]:
from tensorflow.keras import Sequential, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

def load_own_model():

    model = Sequential()

    model.add(layers.Conv2D(16, kernel_size=10, activation='relu', input_shape=(224, 224, 1)))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(8, activation='softmax'))
    
    opt = optimizers.Adam(learning_rate=1e-4)
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model

In [71]:
model_ = load_own_model()
model_.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_19 (Conv2D)          (None, 215, 215, 16)      1616      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 71, 71, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_20 (Conv2D)          (None, 64, 64, 32)        32800     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 21, 21, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_21 (Conv2D)          (None, 16, 16, 32)        36896     
                                                                 
 max_pooling2d_14 (MaxPoolin  (None, 5, 5, 32)       

In [74]:
es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 5, 
                   verbose = 1, 
                   restore_best_weights = True)

history = model_.fit(X_train, y_train,
                             validation_data = (X_val, y_val),
                             batch_size = 16, 
                             epochs = 100, 
                             callbacks=[es])

Epoch 1/100
221/221 [==============================] - 22s 98ms/step - loss: 1.6928 - accuracy: 0.4717 - val_loss: 1.9777 - val_accuracy: 0.2331
Epoch 2/100
221/221 [==============================] - 33s 151ms/step - loss: 1.2930 - accuracy: 0.5433 - val_loss: 1.7677 - val_accuracy: 0.2925
Epoch 3/100
221/221 [==============================] - 28s 126ms/step - loss: 1.1130 - accuracy: 0.6012 - val_loss: 1.6761 - val_accuracy: 0.3789
Epoch 4/100
221/221 [==============================] - 29s 129ms/step - loss: 0.9390 - accuracy: 0.6819 - val_loss: 1.4698 - val_accuracy: 0.4662
Epoch 5/100
221/221 [==============================] - 28s 126ms/step - loss: 0.8083 - accuracy: 0.7157 - val_loss: 1.7691 - val_accuracy: 0.3915
Epoch 6/100
221/221 [==============================] - 28s 126ms/step - loss: 0.6981 - accuracy: 0.7586 - val_loss: 1.3512 - val_accuracy: 0.5500
Epoch 7/100
221/221 [==============================] - 27s 124ms/step - loss: 0.5625 - accuracy: 0.8219 - val_loss: 1.4701 - 

KeyboardInterrupt: 